In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import json
from webdriver_manager.chrome import ChromeDriverManager
from pandas.io.json import json_normalize
import pandas as pd

In [18]:
driver = webdriver.Chrome(ChromeDriverManager().install())

driver.get('https://www.whoscored.com/Matches/1469516/Live/England-FA-Cup-2019-2020-Arsenal-Chelsea')
data = driver.execute_script("return matchCentreData;")

with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)
       
driver.close()

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240
[WDM] - Driver [/Users/ollinj01/.wdm/drivers/chromedriver/mac64/86.0.4240.22/chromedriver] found in cache


In [19]:
home_team, away_team = data["home"]["name"], data["away"]["name"]
with open(f"{home_team}_{away_team}.json", "w") as f: ## save as Arsenal_Liverpool.json in present directory
    json.dump(data, f, indent=4)

In [5]:
# data['events']

In [6]:
# df = json_normalize(data["events"], sep="_")

In [7]:
# df.isGoal.value_counts()

In [8]:
# df['Assist'] = df['isGoal'].shift(-1)

In [9]:
# mask1 = df['Assist'] == True
# df.loc[mask1]

In [10]:
# mask = (df['playerId'] == 239872) & (df['Assist'] == True)
# pepe = df.loc[mask]
# pepe

In [94]:
with open('Arsenal_Bou.json') as json_file:
    data = json.load(json_file)

event = []
minute = []
second = []
player = []
team = []
outcome = []
x = []
y = []
endX = []
endY = []
passtype = []


for each in data['events']:
    event.append(each['type']['displayName'])
    try:
        minute.append(each['minute'])
    except:
        minute.append('')
    try:
        second.append(each['second'])
    except:
        second.append('')
    try:
        player.append(each['playerId'])
    except:
        player.append('')
    try:
        team.append(each['teamId'])
    except:
        team.append('')
    try:
        endX.append(each['endX'])
        endY.append(each['endY'])
    except:
        endX.append('')
        endY.append('')
    outcome.append(each['outcomeType']['displayName'])
    x.append(each['x'])
    y.append(each['y'])

    for key, value in each.items():
        passType = 'OpenPlay'
        if key == 'qualifiers':
            for item in value:
                if item['type']['displayName'] == 'BigChanceCreated':
                    passType = item['type']['displayName']
                if item['type']['displayName'] == 'ShotAssist':
                    passType = item['type']['displayName']
                elif item['type']['displayName'] == 'CornerTaken':
                    passType = item['type']['displayName']
                elif item['type']['displayName'] == 'ThrowIn':
                    passType = item['type']['displayName']
            passtype.append(passType)
                
events = pd.DataFrame({'Event':event, 'Minute':minute, 'Second':second, 'Player':player, 'Team':team,
                       'Outcome':outcome, 'X':x, 'Y':y,'endX':endX, 'endY':endY, 'passtype':passtype})

In [95]:
# data['events']

In [96]:
events.head()

,Event,Minute,Second,Player,Team,Outcome,X,Y,endX,endY,passtype
0,Start,0,0,,13,Successful,0.0,0.0,,,OpenPlay
1,Start,0,0,,107,Successful,0.0,0.0,,,OpenPlay
2,Pass,0,1,131079,107,Successful,50.0,50.0,51.8,51.2,OpenPlay
3,Pass,0,3,344716,107,Successful,47.9,43.4,39.3,30.9,OpenPlay
4,Pass,0,6,127129,107,Successful,39.3,30.9,32.1,61.5,OpenPlay


In [97]:
mask = (events['Player'] == 239872) & (events['passtype'] == 'BigChanceCreated')
pepe = events.loc[mask]
pepe

,Event,Minute,Second,Player,Team,Outcome,X,Y,endX,endY,passtype
1387,Pass,79,52,239872,13,Successful,75.8,76.7,91.9,44.3,BigChanceCreated


In [98]:
mask = (events['Event'] == 'Goal')
goal = events.loc[mask]
goal

,Event,Minute,Second,Player,Team,Outcome,X,Y,endX,endY,passtype
1388,Goal,79,55,80921,13,Successful,92.6,45.5,,,OpenPlay
1576,Goal,90,43,382261,107,Successful,95.4,46.6,,,OpenPlay


In [99]:
events['Assist'] = (events['Event'] == 'Goal').shift(-1)
events['Assist_2nd'] = (events['Event'] == 'Goal').shift(-3)

In [100]:
mask = (events['Player'] == 239872) & (events['Event'] == 'Goal')
pepe_goal = events.loc[mask]
pepe_goal.head()

,Event,Minute,Second,Player,Team,Outcome,X,Y,endX,endY,passtype,Assist,Assist_2nd


In [101]:
mask = (events['Player'] == 239872) & ((events['Assist'] == True) | (events['Assist_2nd'] == True))
pepe_assist = events.loc[mask]
pepe_assist.head()

,Event,Minute,Second,Player,Team,Outcome,X,Y,endX,endY,passtype,Assist,Assist_2nd
1387,Pass,79,52,239872,13,Successful,75.8,76.7,91.9,44.3,BigChanceCreated,True,False


In [102]:
# mask = (events['Player'] == 239872) & (events['Assist_2nd'] == True)
# pepe_assist = events.loc[mask]
# pepe_assist.head()

In [103]:
events.to_csv('Vitoria de Guimaraes_Arsenal.csv', index=False)

In [104]:
import os
import glob
import pandas as pd

In [105]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv( "pepe_arse_all.csv", index=False, encoding='utf-8-sig')